# Mouse Purkinje cell model

This use case shows the steps to select and validate the results obtained after an otimization of a multicomparmental model of mouse Purkinje cell.

- The morphology was taken from neuromorpho.org 

>  (http://neuromorpho.org/neuron_info.jsp?neuron_name=Purkinje-slice-ageP43-6)


- The passive and active properties were taken from the published Purkinje cell model which can be found on modeldb

>  (https://senselab.med.yale.edu/modeldb/ShowModel.cshtml?model=229585)

> Masoli, S., Solinas, S., & D’Angelo, E. (2015). Action potential processing in a detailed Purkinje cell model reveals a critical role for axonal compartmentalization. Frontiers in Cellular Neuroscience, 9(February), 1–22. http://doi.org/10.3389/fncel.2015.00047

- The range of the conductances, to define the range for the optimization, were adapted from:

> Masoli, S., Solinas, S., & D’Angelo, E. (2015). Action potential processing in a detailed Purkinje cell model reveals a critical role for axonal compartmentalization. Frontiers in Cellular Neuroscience, 9(February), 1–22. http://doi.org/10.3389/fncel.2015.00047

> Masoli, S., & D’Angelo, E. (2017). Synaptic Activation of a Detailed Purkinje Cell Model Predicts Voltage-Dependent Control of Burst-Pause Responses in Active Dendrites. Frontiers in Cellular Neuroscience, 11(September), 1–18. http://doi.org/10.3389/fncel.2017.00278

- The validation is based on mouse cell experimental data provided by inlab (SP1) and literature:

> Ransdell, J. L., Dranoff, E., Lau, B., Lo, W. L., Donermeyer, D. L., Allen, P. M., & Nerbonne, J. M. (2017). Erratum: Loss of Navβ4-Mediated Regulation of Sodium Currents in Adult Purkinje Neurons Disrupts Firing and Impairs Motor Coordination and Balance (Cell Reports (2017) 19(3) (532–544) (S2211124717304291) (10.1016/j.celrep.2017.03.068)). Cell Reports, 20(6), 1502. http://doi.org/10.1016/j.celrep.2017.07.072

## Technical Details

The optimization procedure requires a lot of computational power and the available supercomputer (Julia - Julich), accesible from the Unicore system, is not strong enough to do the job in a reasonable amount of time.

## How to run

This notebook can be either run with "Cell" - "Run All", or with "Cell" - "Run Cells" (from the above menu)
- With the first option ("Run All"), the entire notebook will be executed till the end.
- With the second option ("Run Cells"), only the code of a single cell will be executed. Clicking the following symbol ![skip-next](https://raw.githubusercontent.com/antonelepfl/testvue/master/skip_next.png) will execute the notebook cell-by-cell, giving the possibility to interfere with the configuration in each cell.


Each time a notebook is executed, a kernel status symbol is displayed in the top right corner of the notebook
- When there is a circle bullet •, it means that the kernel is running and the user have to be sure to not interfere with the code execution
- When there is an empty circle bullet ○, it means that the kernel is idle and the user can interfere with the notebook.

More details can be taken from the [Guidebook](https://lbologna.github.io/hbp-sp6-guidebook/)


## Preparation
The following cells will setup the access to the Collab Storage and download the required morphologies and ion channels.

In [ ]:
!ln -sfn /home/jovyan/.local/nrn-7.6/ /home/jovyan/.local/nrn

In [ ]:
!pip3 install --upgrade --no-dependencies "hbp-service-client==1.1.1" >/dev/null

In [ ]:
#access to the collaboratory storage
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()

This cell is used to prevent interference from previous runs of this notebooks

In [ ]:
%%capture
!rm *step* 
!rm *indiv*
!rm -r valid
!rm Purkinje_res.zip
!rm Purkinje_multi.zip

Downloading morphology, ionic channels, previosly optimized results to the location where this notebooks runs.

In [ ]:
collab_storage_path = '/9135'

clients.storage.download_file(collab_storage_path + '/Single cells usecases/Purkinje cell multi compartmental/Purkinje_multi.zip', 'Purkinje_multi.zip')
clients.storage.download_file(collab_storage_path + '/Single cells usecases/Purkinje cell multi compartmental/Purkinje_res.zip', 'Purkinje_res.zip')

In [ ]:
%%capture
!unzip -o Purkinje_multi.zip
!unzip -o Purkinje_res.zip

Modules used for the extraction of the features from voltage time traces and to plot them

In [ ]:
import collections
import numpy as np
import efel
import matplotlib.pyplot as plt
import subprocess

feature_values = collections.defaultdict(dict)
conductvalues = np.genfromtxt("final_pop.txt")

## Plot of the traces belonging to a single individual

After executing the next cell you can select the number of the sim to be plotted.

In [ ]:
import ipywidgets as widgets
import numpy as np
conductvalues = np.genfromtxt("final_pop.txt")


sim_number = widgets.BoundedIntText(
    value=0,
    min=0,
    max=len(conductvalues),
    step=1,
    description='Sim num:',
    disabled=False
)
display(sim_number)

Plot the selected sim

In [ ]:
import os

# Location of the recording. For the Purkinje cell there is only one.
location = '' 

steps = {
    1: "spontaneuos firing",
    2: "0.5nA current injection",
    3: "1nA current injection",
}

for current_inj_step in range (1,4): #Number of the current step used in the optimization. 1, first step, 2 second step and so on.
    filetxt = str(sim_number.value) + '_sim_step' + str(current_inj_step) + location +'.txt'
    if os.path.isfile(filetxt) == False:
        print( 'file not found')
        pass
    
    elif os.path.isfile(filetxt):    
        trace = np.genfromtxt(filetxt)
        
        time, voltage = zip(*trace)
        
        fig, ax = plt.subplots()
        ax.plot(time,voltage, 'b', label='spikes, %s' % (steps[current_inj_step]))

        legend = ax.legend(loc='upper right', shadow=True)

        frame = legend.get_frame()
        frame.set_facecolor('0.90')

        plt.xlabel("Time (ms)")
        plt.ylabel("Membrane voltage (mv) ")

## Defining some functions to automatically select, analyze and save the results from the voltage traces.
The selection process is based on the frequency, calculated with the ISI values, for three steps:

- Spontaneuos firing
- 0.5nA positive current injection
- 1nA positive current injection


In [ ]:
def trace_features(trace_init, trace_end, indiv, stepindex, location): 
    
    filename = str(indiv) + '_sim_step' + str(stepindex) + location + '.txt'
    print( "trace number", indiv)
    
    print( filename)
    fileeps = str(indiv) + '_sim_step' + str(stepindex) + '_window' + location +'.eps'
    if os.path.isfile(filename) and os.path.isfile(fileeps):
        print( filename, 'trovato')
        pass
    
    elif os.path.isfile(filename):
        trace = np.genfromtxt(filename)

        time = []
        voltage = []

        for x in range(len(trace)):
            time.append(trace[x,0])
            voltage.append(trace[x,1])
        
        fig, ax = plt.subplots()
        ax.plot(time, voltage, 'b', label='spikes')

        legend = ax.legend(loc='upper right', shadow=True)

        frame = legend.get_frame()
        frame.set_facecolor('0.90')

        plt.xlabel("time (ms)")
        plt.ylabel("membrane voltage (mv) ")

        plt.savefig('feat_indiv_' + str(indiv) + '_sim_step' + str(stepindex) +  '_start_'+ str(trace_init) + '_end_'+ str(trace_end) + '_window' + location + '.eps')
        plt.close()
        
        traces = []

        trace = {}  
        trace['T'] = time
        trace['V'] = voltage
        trace['stim_start'] = [trace_init]
        trace['stim_end'] = [trace_end]
        traces = [trace]  

        result = efel.getFeatureValues(traces, ['mean_frequency',
                                                'Spikecount',
                                                'ISI_CV'])
    
        for feature_name, feature_list in result[0].items():
            if feature_list is not None and len(feature_list) > 0:
                feature_values[feature_name] = np.mean(feature_list)
            else:
                feature_values[feature_name] = 0

                
        filename = 'feat_indiv_' + str(indiv) + '_sim_step' + str(stepindex) + '_start_'+ str(trace_init) + '_end_'+ str(trace_end) + '_window' + location + '.csv'
        fo = open(filename, "w")
        for e,v in feature_values.items():
            fo.write(e+ ' ' + str(v)+"\n")    
        fo.close() 
        
def save_feat_name(trace_init, trace_end, indiv, stepindex, location):
    global final_name_matrix
    filename = 'feat_indiv_' + str(indiv) + '_sim_step' + str(stepindex) + '_start_'+ str(trace_init) + '_end_'+ str(trace_end) + '_window' + location + '.csv'
    trace = np.genfromtxt(filename, dtype = [('feat_name','S20'),('value','f8')])

    name = []
    for x in trace:
        name.append(x['feat_name'])

    trace_name = np.matrix(name)
    add_number_gen = ['Indiv_' + str(indiv)]
    trace_indiv = np.matrix(add_number_gen)
    x_size,y_size = trace_name.shape
    trace_name_mod = np.reshape(trace_name,(y_size ,1)) 
    final_name_matrix = np.append(trace_indiv, trace_name_mod, axis=0)  

def matrix_builder(filename, stepindex,location):
    global final_value_matrix  
    trace = np.genfromtxt(filename, dtype = [('feat_name','S20'),('value','f8')])
    
    name = []
    value = []
    for x in trace:
        value.append(x['value'])

#value of each feature
    trace_value = np.matrix(value)
    
    if location == '':
        location = '_soma'
    else:
        pass
    
    stepindex_ext = [location + '_step_' +  str(stepindex)]
    
    trace_step = np.matrix(stepindex_ext)
    
    x_size,y_size = trace_value.shape
    trace_value_mod = np.reshape(trace_value,(y_size ,1))

    final_value_matrix = np.append(trace_step, trace_value_mod, axis=0)    


def trace_load(trace_init, trace_end, indiv, stepindex, location): 
    global base_matrix_value
    filename = 'feat_indiv_' + str(indiv) + '_sim_step' + str(stepindex) + '_start_'+ str(trace_init) + '_end_'+ str(trace_end) + '_window' + location + '.csv'
    if os.path.isfile(filename) == True:
        matrix_builder(filename,stepindex, location)
        base_matrix_value = np.append(base_matrix_value, final_value_matrix, axis=1)
        base_matrix_value[0,0] =  'Indiv_' + str(indiv)
        print( 'done')
        np.savetxt('Feature'+ '_indiv_' + str(indiv) + '_start_'+ str(trace_init) + '_end_'+ str(trace_end) + '.csv', base_matrix_value, delimiter = ',', fmt="%10s")
    
    if os.path.isfile(filename) == False:
        print( filename, 'not found')
        pass

In [ ]:
def selector(indiv):
    threshold = {
        1: 0.05,
        2: 0.3,
        3: 0.36
    }
    
    
    if base_matrix_value.shape <= (3,3):
        print( 'matrix not valid')
    else:     
            
        realy_valid = []
        realy_bad = []
                         
        for i in range(1,4):   
            
            ISI = float(base_matrix_value[3,i])
            
            if ISI <= threshold[i]:
                realy_valid.append(i)
            if ISI >= threshold[i]:
                realy_bad.append(i)
                

                
        localdir = os.getcwd()
        #print localdir
        if len(realy_valid) == 3:
            print( 'Physiological Purkinje cell')
            print( 'indiv', indiv)
            
            if os.path.isdir(localdir +"/valid") == False:
                os.makedirs(localdir +"/valid")
                
                actualdir = localdir + "/valid"
                indiv_number = str(indiv) + '_'
                indiv_number_feat_single = "feat_indiv_" + str(indiv) + '_'
                indiv_number_feat_matrix = "Feature_indiv_" + str(indiv)  + '_'
                
                if indiv == 0:
                    pass
                else:
                    subprocess.call("mv " + indiv_number + "* " + actualdir, shell=True)
                    subprocess.call("mv " + indiv_number_feat_single + "* " + actualdir, shell=True)
                    subprocess.call("mv " + indiv_number_feat_matrix + "* " + actualdir, shell=True)
                
            elif os.path.isdir(localdir +"/valid") == True:
                print( 'dir exist!', "/valid")
                print( 'indiv', indiv)
                
                actualdir = localdir + "/valid"
                indiv_number = str(indiv) + '_'
                indiv_number_feat_single = "feat_indiv_" + str(indiv) + '_'
                indiv_number_feat_matrix = "Feature_indiv_" + str(indiv) + '_'
                                
                if indiv == 0:
                    pass
                else:
                    subprocess.call("mv " + indiv_number + "* " + actualdir, shell=True)
                    subprocess.call("mv " + indiv_number_feat_single + "* " + actualdir, shell=True)
                    subprocess.call("mv " + indiv_number_feat_matrix + "* " + actualdir, shell=True)     
        

In [ ]:
!mkdir -p valid

The following cell starts the process to:
- extract of the features from the voltage-traces generated from the last iteration of the optimization.
- Each individual properties are save together in a series of files.
- Each individual is validated against the ISI values obtained from SP1 data and literature (Ransdell and Narbonne, 2017).
- The individual that are within the parameters are moved to the "good" directory. 

In [ ]:
soma = ''

for first_step in range(1,4):
    trace_features(0,2100,0,first_step,soma)
    
save_feat_name(0,2100,0,1,soma)

base_matrix_value = final_name_matrix

for indiv in range(0,len(conductvalues)):
    for step in range(1,4): #number of steps used during the optimization procedure.
            trace_features(0,2100,indiv,step,soma)  
            trace_load(0,2100,indiv,step,soma)  
    selector(indiv)
    base_matrix_value = final_name_matrix        

This cell reads the valid results and show them as a series of numbers

In [ ]:
valid_dir = "/home/jovyan/valid/"
good_results = []
for i in range(len(conductvalues)):
    filename_final = valid_dir + str(i) + '_sim_step1' + '.txt'
    fileeps_final = valid_dir + str(i) + '_sim_step1' +'.eps'
    if os.path.isfile(filename_final) == True:
        good_results.append(i)
        
print( good_results        )

Compiling the mod files for each ionic channel.

In [ ]:
!nrnivmodl >/dev/null 2>/dev/null

The following two cells contain the same code, used during the optimization, modified to simulate the valid results.

The following class (evoked by "do_replace_axon = true" in the following cell) overwrite the one provided with BluePyOpt and permits to define the properties and quantity of the sections composing the axon. This is due the fact that Purkinje cells have a myelinated axon composed by Ranvier Nodes and myelinated sections.

In [ ]:
import bluepyopt as bpop
import bluepyopt.ephys as ephys	
import math

class nrnPCmorphology(ephys.morphologies.NrnFileMorphology):
  def __init__(self, morphology_path, do_replace_axon):
    super(nrnPCmorphology, self).__init__(morphology_path, do_replace_axon)

  @staticmethod
  def replace_axon(sim=None, icell=None):
        """Replace axon"""
        ais_diams = [icell.axon[0].diam, icell.axon[0].diam]

        # Define origin of distance function
        sim.neuron.h.distance(sec=icell.soma[0])
        for section in icell.axonal:
            # If distance to soma is larger than 60, store diameter
            if sim.neuron.h.distance(0.5, sec=section) > 60:
                ais_diams[1] = section.diam
                break

        for section in icell.axonal:
            sim.neuron.h.delete_section(sec=section)

        # Create new axon array
        sim.neuron.h.execute('create axon[11]', icell)

        icell.axon[0].nseg = 1
        icell.axon[0].L = 17
        icell.axon[0].diam = 0.97
        
        icell.axon[1].nseg = 1
        icell.axon[1].L = 4
        icell.axon[1].diam = 0.97
        
        icell.axon[2].nseg = 1
        icell.axon[2].L = 100
        icell.axon[2].diam = 0.73
        
        icell.axon[3].nseg = 1
        icell.axon[3].L = 4
        icell.axon[3].diam = 0.73
        
        icell.axon[4].nseg = 1
        icell.axon[4].L = 100
        icell.axon[4].diam = 0.73
        
        icell.axon[5].nseg = 1
        icell.axon[5].L = 4
        icell.axon[5].diam = 0.73
        
        icell.axon[6].nseg = 1
        icell.axon[6].L = 100
        icell.axon[6].diam = 0.73
        
        icell.axon[7].nseg = 1
        icell.axon[7].L = 4
        icell.axon[7].diam = 0.73
        
        icell.axon[8].nseg = 1
        icell.axon[8].L = 100
        icell.axon[8].diam = 0.73
        
        for i in range(0,11):
            icell.axonal.append(sec = icell.axon[i])
            icell.all.append(sec = icell.axon[i])

        icell.axon[0].connect(icell.soma[0], 1.0, 0.0)
        icell.axon[1].connect(icell.axon[0], 1.0, 0.0)
        icell.axon[2].connect(icell.axon[1], 1.0, 0.0)
        icell.axon[3].connect(icell.axon[2], 1.0, 0.0)
        icell.axon[4].connect(icell.axon[3], 1.0, 0.0)
        icell.axon[5].connect(icell.axon[4], 1.0, 0.0)
        icell.axon[6].connect(icell.axon[5], 1.0, 0.0)
        icell.axon[7].connect(icell.axon[6], 1.0, 0.0)
        icell.axon[8].connect(icell.axon[7], 1.0, 0.0)


        icell.AIS.append(sec = icell.axon[0])
        icell.AISK.append(sec = icell.axon[1]) 
        icell.myelin0.append(sec = icell.axon[2]) 
        icell.rn0.append(sec = icell.axon[3]) 
        icell.myelin1.append(sec = icell.axon[4]) 
        icell.rn1.append(sec = icell.axon[5]) 
        icell.myelin2.append(sec = icell.axon[6]) 
        icell.rn2.append(sec = icell.axon[7]) 
        icell.myelin3.append(sec = icell.axon[8]) 


        for section in icell.dend:
            if section.diam >= 0 and section.diam <= 6:
                icell.dendall.append(sec=section)
            if section.diam >= 1.4 and section.diam <= 6:
                icell.dendtrunk.append(sec=section)
            if section.diam >= 2.05 and section.diam <= 6:  
                icell.dendsodium.append(sec=section)
            if section.diam >= 0 and section.diam <= 1.4:
                section.cm = ((11.510294 * math.exp( - 1.376463 * section.diam) + 2.120503))*1.5
       
        sim.neuron.h.celsius = 37

Purkinje cell are valid when, in the absence of sodium channel in the the AIS, the cell is unable to generate spontaneuos firing.

The numbers printed in the output of this cell refers to the simulation of each results validated in the previous part of the usecase.

***Warning***: **The next step can take up to 15m to be perfomed.**



In [ ]:
#Purkinje cell optimization with BluePyOpt

#import matplotlib.pyplot as plt
import os
import numpy as np

#Morphology location
morph = nrnPCmorphology('purkinje_mouse.swc', do_replace_axon = True)

#Locations

dend_all = ephys.locations.NrnSeclistLocation('dendall', seclist_name='dendall')
dend_trunk = ephys.locations.NrnSeclistLocation('dendtrunk', seclist_name='dendtrunk')
dend_sodium = ephys.locations.NrnSeclistLocation('dendsodium', seclist_name='dendsodium')

somatic_loc = ephys.locations.NrnSeclistLocation('somatic', seclist_name='somatic')

axon0 = ephys.locations.NrnSeclistLocation('AIS', seclist_name='AIS')
axon1 = ephys.locations.NrnSeclistLocation('AISK', seclist_name='AISK')
axon2 = ephys.locations.NrnSeclistLocation('myelin0', seclist_name='myelin0')
axon3 = ephys.locations.NrnSeclistLocation('rn0', seclist_name='rn0')
axon4 = ephys.locations.NrnSeclistLocation('myelin1', seclist_name='myelin1')
axon5 = ephys.locations.NrnSeclistLocation('rn1', seclist_name='rn1')
axon6 = ephys.locations.NrnSeclistLocation('myelin2', seclist_name='myelin2')
axon7 = ephys.locations.NrnSeclistLocation('rn2', seclist_name='rn2')
axon8 = ephys.locations.NrnSeclistLocation('myelin3', seclist_name='myelin3')

#Channels list

#DENDRITES.

#Passive properties - Frozen for single parameters and not frozen for ranges of parameters ie channels.
cm_dend_all = ephys.parameters.NrnSectionParameter(
        name='cm_dend_all', #- must be unique
        param_name = 'cm',
        value = 1,
        locations=[dend_trunk, dend_sodium],
        frozen=True)

cm_soma = ephys.parameters.NrnSectionParameter(
        name='cm_soma', #- must be unique
        param_name='cm',
        value=0.77,
        locations=[somatic_loc],
        frozen=True)

cm_rn_only = ephys.parameters.NrnSectionParameter(
        name='cm_axon_rn', #- must be unique
        param_name='cm',
        value=0.77,
        locations=[axon0, axon1, axon3, axon5, axon7],
        frozen=True)

cm_myelin_only = ephys.parameters.NrnSectionParameter(
        name='cm_axon_myelin', #- must be unique
        param_name='cm',
        value=1.87e-11,
        locations=[axon2, axon4, axon6, axon8],
        frozen=True)


ena = ephys.parameters.NrnSectionParameter(
        name='ena_all', #- must be unique
        param_name='ena',
        value=80,
        locations=[dend_sodium, somatic_loc, axon0, axon3, axon5, axon7],
        frozen=True)

ek = ephys.parameters.NrnSectionParameter(
        name='ek_all', #- must be unique
        param_name='ek',
        value=-88,
        locations=[dend_all, somatic_loc, axon0, axon1, axon3, axon5, axon7],
        frozen=True)

eleak = ephys.parameters.NrnSectionParameter(
        name='eleak_all', #- must be unique
        param_name='e_Leak',
        value = -61,
        locations=[dend_all, somatic_loc, axon0, axon1, axon3, axon5, axon7],
        frozen=True)

leak_soma = ephys.parameters.NrnSectionParameter(
        name='leak_soma', #- must be unique
        param_name='gmax_Leak',
        value = 1E-3,
        locations=[somatic_loc],
        frozen=True)

leak_rest = ephys.parameters.NrnSectionParameter(
        name='leak_rest', #- must be unique
        param_name='gmax_Leak',
        value = 0.0003,
        locations=[dend_all, axon0, axon1, axon3, axon5, axon7],
        frozen=True)

eca = ephys.parameters.NrnSectionParameter(
        name='eca_all', #- must be unique
        param_name='eca',
        value = 137.5,
        locations=[dend_all, somatic_loc, axon0, axon3, axon5, axon7],
        frozen=True)

Ra = ephys.parameters.NrnSectionParameter(
        name='ra_all', #- must be unique
        param_name='Ra',
        value = 122,
        locations=[dend_all, somatic_loc, axon0, axon1, axon2, axon3, axon4, axon5, axon6, axon7, axon8],
        frozen=True)

gpas_myelin_only = ephys.parameters.NrnSectionParameter(
        name='g_pas_axon_myelin', #- must be unique
        param_name='g_pas',
        value=5e-8,
        locations=[axon2, axon4, axon6, axon8],
        frozen=True)

epas_myelin_only = ephys.parameters.NrnSectionParameter(
        name='e_pas_axon_myelin', #- must be unique
        param_name='e_pas',
        value=-63,
        locations=[axon2, axon4, axon6, axon8],
        frozen=True)

ca_pump_dend = ephys.parameters.NrnSectionParameter(
        name='TotalPump_dend', 
        param_name='TotalPump_cdp5',
        value=5e-8,
        locations=[dend_all],
        frozen=True)

ca_pump_soma = ephys.parameters.NrnSectionParameter(
        name='TotalPump_soma', 
        param_name='TotalPump_cdp5',
        value=2e-8,
        locations=[somatic_loc, axon0],
        frozen=True)

ca_pump_axon = ephys.parameters.NrnSectionParameter(
        name='TotalPump_axon', 
        param_name='TotalPump_cdp5',
        value=5e-7,
        locations=[axon3, axon5, axon7],
        frozen=True)

#CHANNELS
#DEND

#Mixed
PC_Leak = ephys.mechanisms.NrnMODMechanism(
        name='Leak_all',
        prefix='Leak',
        locations=[dend_all, somatic_loc, axon0, axon1, axon3, axon5, axon7])

PC_Nav16 = ephys.mechanisms.NrnMODMechanism(
        name='Nav1_6',
        prefix='Nav1_6',
        locations=[dend_sodium, somatic_loc, axon0, axon3, axon5, axon7])

PC_Kv1_1 = ephys.mechanisms.NrnMODMechanism(
        name='Kv1_1',
        prefix='Kv1_1',
        locations=[dend_all, somatic_loc, axon1])

PC_Kv1_5 = ephys.mechanisms.NrnMODMechanism(
        name='Kv1_5',
        prefix='Kv1_5',
        locations=[dend_all])

PC_Kv3_3 = ephys.mechanisms.NrnMODMechanism(
        name='Kv3_3',
        prefix='Kv3_3',
        locations=[dend_all])

PC_Kv3_4 = ephys.mechanisms.NrnMODMechanism(
        name='Kv3_4',
        prefix='Kv3_4',
        locations=[somatic_loc, axon0, axon3, axon5, axon7])

PC_Kv4_3 = ephys.mechanisms.NrnMODMechanism(
        name='Kv4_3',
        prefix='Kv4_3',
        locations=[dend_all])

PC_Kir2_3 = ephys.mechanisms.NrnMODMechanism(
        name='Kir2_3',
        prefix='Kir2_3',
        locations=[somatic_loc, dend_trunk])

PC_Kca1_1 = ephys.mechanisms.NrnMODMechanism(
        name='Kca1_1',
        prefix='Kca1_1',
        locations=[dend_all, somatic_loc])

PC_Kca2_2 = ephys.mechanisms.NrnMODMechanism(
        name='Kca2_2',
        prefix='Kca2_2',
        locations=[dend_trunk, somatic_loc])

PC_Kca3_1 = ephys.mechanisms.NrnMODMechanism(
        name='Kca3_1',
        prefix='Kca3_1',
        locations=[dend_trunk, somatic_loc])

PC_Cav2_1 = ephys.mechanisms.NrnMODMechanism(
        name='Cav2_1',
        prefix='Cav2_1',
        locations=[dend_all, somatic_loc, axon0, axon3, axon5, axon7])

PC_Cav3_1 = ephys.mechanisms.NrnMODMechanism(
        name='Cav3_1',
        prefix='Cav3_1',
        locations=[dend_trunk, somatic_loc, axon0, axon3, axon5, axon7])

PC_Cav3_2 = ephys.mechanisms.NrnMODMechanism(
        name='Cav3_2',
        prefix='Cav3_2',
        locations=[dend_trunk, somatic_loc])

PC_Cav3_3 = ephys.mechanisms.NrnMODMechanism(
        name='Cav3_3',
        prefix='Cav3_3',
        locations=[dend_all, somatic_loc])

PC_HCN1 = ephys.mechanisms.NrnMODMechanism(
        name='HCN1',
        prefix='HCN1',
        locations=[dend_all, somatic_loc])

PC_cdp5 = ephys.mechanisms.NrnMODMechanism(
        name='cdp5',
        prefix='cdp5',
        locations=[dend_all, somatic_loc, axon0, axon3, axon5, axon7])

PC_pass = ephys.mechanisms.NrnMODMechanism(
        name='pas',
        prefix='pas',
        locations=[axon2, axon4, axon6, axon8])

#CHANNELS PARAM

#DEND_all
PC_Kv1_1_param_dend_all = ephys.parameters.NrnSectionParameter(
        name='gbar_Kv1_1_dend_all',
        param_name='gbar_Kv1_1',
        bounds=[0.001, 0.0015],
        locations=[dend_all],
        frozen=False)

PC_Kv1_5_param_dend_all = ephys.parameters.NrnSectionParameter(
        name='gKur_Kv1_5_dend_all',
        param_name='gKur_Kv1_5',
        bounds=[0.8e-4, 1.5e-4],
        locations=[dend_all],
        frozen=False)

PC_Kv3_3_param_dend_all = ephys.parameters.NrnSectionParameter(                                    
        name='gbar_Kv3_3_dend_all',
        param_name='gbar_Kv3_3',
        locations=[dend_all],
        bounds=[0.009, 0.015],
        frozen=False) 

PC_Kv4_3_param_dend_all = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv4_3_dend_all',
        param_name='gkbar_Kv4_3',
        locations=[dend_all],
        bounds=[0.0008, 0.0015],
        frozen=False) 

PC_Kca1_1_param_dend_all = ephys.parameters.NrnSectionParameter(
        name='gbar_Kca1_1_dend_all',
        param_name='gbar_Kca1_1',
        bounds=[2e-2, 5e-2],
        locations=[dend_all],
        frozen=False)

PC_Cav2_1_param_dend_all = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav2_1_dend_all',
        param_name='pcabar_Cav2_1',
        bounds=[2.5e-3, 4.5e-3],
        locations=[dend_all],
        frozen=False)

PC_Cav3_3_param_dend_all = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav3_3_dend_all',
        param_name='pcabar_Cav3_3',
        bounds=[0.00008, 0.0002],
        locations=[dend_all],
        frozen=False)

PC_HCN1_param_dend_all = ephys.parameters.NrnSectionParameter(
        name='gbar_HCN1_dend_all',
        param_name='gbar_HCN1',
        bounds=[0.000002, 0.000005],
        locations=[dend_all],
        frozen=False)


#dend_trunk
PC_Kir2_3_param_dend_trunk = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kir2_3_dend_trunk',
        param_name='gkbar_Kir2_3',
        locations=[dend_trunk],
        bounds=[0.000008, 0.00002],
        frozen=False)

PC_Kca2_2_param_dend_trunk = ephys.parameters.NrnSectionParameter(
        name='gkbar_Kca2_2_dend_trunk',
        param_name='gkbar_Kca2_2',
        bounds=[7e-6, 2e-3],
        locations=[dend_trunk],
        frozen=False)

PC_Kca3_1_param_dend_trunk = ephys.parameters.NrnSectionParameter(
        name='gkbar_Kca3_1_dend_trunk',
        param_name='gkbar_Kca3_1',
        bounds=[0.0015, 0.004],
        locations=[dend_trunk],
        frozen=False)

PC_Cav3_1_param_dend_all = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav3_1_dend_all',
        param_name='pcabar_Cav3_1',
        bounds=[3e-6, 7e-6],
        locations=[dend_trunk],
        frozen=False)

PC_Cav3_2_param_dend_all = ephys.parameters.NrnSectionParameter(
        name='gcabar_Cav3_2_dend_all',
        param_name='gcabar_Cav3_2',
        bounds=[0.00095, 0.002],
        locations=[dend_trunk],
        frozen=False)

#dend_sodium
PC_Nav16_param_dend_sodium = ephys.parameters.NrnSectionParameter(                                    
        name='gbar_Nav1_6_dend_sodium',
        param_name='gbar_Nav1_6',
        locations=[dend_sodium],
        bounds=[0.013, 0.017],
        frozen=False)  

#SOMA
PC_Nav16_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gbar_Nav1_6_soma',
        param_name='gbar_Nav1_6',
        locations=[somatic_loc],
        bounds=[0.185, 0.21],
        frozen=False)  

PC_Kv1_1_param_soma = ephys.parameters.NrnSectionParameter(
        name='gbar_Kv1_1_soma',
        param_name='gbar_Kv1_1',
        bounds=[0.001, 0.003],
        locations=[somatic_loc],
        frozen=False)

PC_Kv3_4_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv3_4_soma',
        param_name='gkbar_Kv3_4',
        locations=[somatic_loc],
        bounds=[0.04, 0.07],
        frozen=False) 

PC_Kir2_3_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kir2_3_soma',
        param_name='gkbar_Kir2_3',
        locations=[somatic_loc],
        bounds=[0.00001, 0.00007],
        frozen=False)

PC_Kca1_1_param_soma = ephys.parameters.NrnSectionParameter(
        name='gbar_Kca1_1_soma',
        param_name='gbar_Kca1_1',
        bounds=[0.008, 0.015],
        locations=[somatic_loc],
        frozen=False)

PC_Kca2_2_param_soma = ephys.parameters.NrnSectionParameter(
        name='gkbar_Kca2_2_soma',
        param_name='gkbar_Kca2_2',
        bounds=[8e-4, 1.5e-3],
        locations=[somatic_loc],
        frozen=False)

PC_Kca3_1_param_soma = ephys.parameters.NrnSectionParameter(
        name='gkbar_Kca3_1_soma',
        param_name='gkbar_Kca3_1',
        bounds=[0.008, 0.015],
        locations=[somatic_loc],
        frozen=False)

PC_Cav2_1_param_soma = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav2_1_soma',
        param_name='pcabar_Cav2_1',
        bounds=[1.5e-4, 3e-4],
        locations=[somatic_loc],
        frozen=False)

PC_Cav3_1_param_soma = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav3_1_soma',
        param_name='pcabar_Cav3_1',
        bounds=[4e-6, 9e-6],
        locations=[somatic_loc],
        frozen=False)

PC_Cav3_2_param_soma = ephys.parameters.NrnSectionParameter(
        name='gcabar_Cav3_2_soma',
        param_name='gcabar_Cav3_2',
        bounds=[0.0006, 0.001],
        locations=[somatic_loc],
        frozen=False)

PC_Cav3_3_param_soma = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav3_3_soma',
        param_name='pcabar_Cav3_3',
        bounds=[0.000095, 0.0004],
        locations=[somatic_loc],
        frozen=False)

PC_HCN1_param_soma = ephys.parameters.NrnSectionParameter(
        name='gbar_HCN1_soma',
        param_name='gbar_HCN1',
        bounds=[0.0008, 0.002],
        locations=[somatic_loc],
        frozen=False)

#AXON
#AIS
PC_Nav16_param_axon0 = ephys.parameters.NrnSectionParameter(                                    
        name='gbar_Nav1_6_axon0',
        param_name='gbar_Nav1_6',
        locations=[axon0],
        bounds=[0, 0.7],
        frozen=False)  

PC_Kv3_4_param_axon0 = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv3_4_axon0',
        param_name='gkbar_Kv3_4',
        locations=[axon0],
        bounds=[0.009, 0.02],
        frozen=False) 

PC_Cav2_1_param_axon0 = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav2_1_axon0',
        param_name='pcabar_Cav2_1',
        bounds=[1e-4, 4e-4],
        locations=[axon0],
        frozen=False)

PC_Cav3_1_param_axon0 = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav3_1_axon0',
        param_name='pcabar_Cav3_1',
        bounds=[5e-6, 1e-5],
        locations=[axon0],
        frozen=False)

#AISK
PC_Kv1_1_param_axon1 = ephys.parameters.NrnSectionParameter(                                    
        name='gbar_Kv1_1_axon1',
        param_name='gbar_Kv1_1',
        locations=[axon1],
        bounds=[0.008, 0.02],
        frozen=False)  

#RN
PC_Nav16_param_axon = ephys.parameters.NrnSectionParameter(                                    
        name='gbar_Nav1_6_axon',
        param_name='gbar_Nav1_6',
        locations=[axon3, axon5, axon7],
        bounds=[0.025, 0.04],
        frozen=False)  

PC_Kv3_4_param_axon = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv3_4_axon',
        param_name='gkbar_Kv3_4',
        locations=[axon3, axon5, axon7],
        bounds=[0.015, 0.03],
        frozen=False) 

PC_Cav2_1_param_axon = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav2_1_axon',
        param_name='pcabar_Cav2_1',
        bounds=[1e-4, 3e-4],
        locations=[axon3, axon5, axon7],
        frozen=False)

PC_Cav3_1_param_axon = ephys.parameters.NrnSectionParameter(
        name='pcabar_Cav3_1_axon',
        param_name='pcabar_Cav3_1',
        bounds=[1e-5, 2e-5],
        locations=[axon3, axon5, axon7],
        frozen=False)

PC2018 = ephys.models.CellModel(
        name='Purkinje2018StefanoMasoli',
        morph=morph,
        mechs=[PC_Leak,
           PC_pass,
           PC_Nav16, 
           PC_Kv1_1,
           PC_Kv1_5,
           PC_Kv3_3,        
           PC_Kv3_4, 
           PC_Kv4_3,
           PC_Kir2_3,       
           PC_Kca1_1,
           PC_Kca2_2,   
           PC_Kca3_1,     
           PC_Cav2_1,
           PC_Cav3_1,
           PC_Cav3_2,
           PC_Cav3_3,
           PC_HCN1,
           PC_cdp5],

        params=[cm_dend_all,
        cm_soma,
        cm_rn_only,
        cm_myelin_only,
        ena,
        ek,
        eleak,
        leak_soma,
        leak_rest,
        eca,
        Ra,
        gpas_myelin_only,
        epas_myelin_only,
        ca_pump_dend,
        ca_pump_soma,
        ca_pump_axon,
        PC_Nav16_param_dend_sodium,
        PC_Kv1_1_param_dend_all,
        PC_Kv1_5_param_dend_all,
        PC_Kv3_3_param_dend_all,
        PC_Kv4_3_param_dend_all,
        PC_Kir2_3_param_dend_trunk,
        PC_Cav2_1_param_dend_all,
        PC_Cav3_1_param_dend_all,
        PC_Cav3_2_param_dend_all,
        PC_Cav3_3_param_dend_all,
        PC_Kca1_1_param_dend_all,
        PC_Kca2_2_param_dend_trunk,
        PC_Kca3_1_param_dend_trunk,
        PC_HCN1_param_dend_all,
        PC_Nav16_param_soma,
        PC_Kv1_1_param_soma,
        PC_Kv3_4_param_soma,
        PC_Kir2_3_param_soma,
        PC_Cav2_1_param_soma,
        PC_Cav3_1_param_soma,
        PC_Cav3_2_param_soma,
        PC_Cav3_3_param_soma,
        PC_Kca1_1_param_soma,
        PC_Kca2_2_param_soma,
        PC_Kca3_1_param_soma,
        PC_HCN1_param_soma,
        PC_Nav16_param_axon0,
        PC_Kv3_4_param_axon0,
        PC_Cav2_1_param_axon0,
        PC_Cav3_1_param_axon0,
        PC_Kv1_1_param_axon1,
        PC_Nav16_param_axon,
        PC_Kv3_4_param_axon,
        PC_Cav2_1_param_axon,
        PC_Cav3_1_param_axon])  

PC2018.seclist_names.append('dendall')
PC2018.seclist_names.append('dendtrunk')
PC2018.seclist_names.append('dendsodium')

PC2018.seclist_names.append('AIS')
PC2018.seclist_names.append('AISK')
PC2018.seclist_names.append('myelin0')
PC2018.seclist_names.append('rn0')
PC2018.seclist_names.append('myelin1')
PC2018.seclist_names.append('rn1')
PC2018.seclist_names.append('myelin2')
PC2018.seclist_names.append('rn2')
PC2018.seclist_names.append('myelin3')

print( PC2018)


soma_loc = ephys.locations.NrnSeclistCompLocation(
        name='soma',
        seclist_name='somatic',
        sec_index=0,
        comp_x=0.5)


sweep_protocols = []
for protocol_name, amplitude in [('step1', 0)]:
    stim = ephys.stimuli.NrnSquarePulse(
                step_amplitude=amplitude,
                step_delay=100,
                step_duration=1000,
                location=soma_loc,
                total_duration=1000)
    rec = ephys.recordings.CompRecording(
            name='%s.soma.v' % protocol_name,
            location=soma_loc,
            variable='v')
    protocol = ephys.protocols.SweepProtocol(protocol_name, [stim], [rec])
    sweep_protocols.append(protocol)
threestep_protocol = ephys.protocols.SequenceProtocol('twostep', protocols=sweep_protocols)

nrn = ephys.simulators.NrnSimulator()


for i in (good_results):
    print( 'Simulation of validated individual:',i)
    default_params = {'gbar_Nav1_6_dend_sodium' : conductvalues[i,0],
              'gbar_Kv1_1_dend_all': conductvalues[i,1], 
              'gKur_Kv1_5_dend_all' : conductvalues[i,2],
              'gbar_Kv3_3_dend_all' : conductvalues[i,3],
              'gkbar_Kv4_3_dend_all' : conductvalues[i,4],
              'gkbar_Kir2_3_dend_trunk' : conductvalues[i,5],
              'pcabar_Cav2_1_dend_all' : conductvalues[i,6],
              'pcabar_Cav3_1_dend_all' : conductvalues[i,7],
              'gcabar_Cav3_2_dend_all' : conductvalues[i,8],
              'pcabar_Cav3_3_dend_all' : conductvalues[i,9],
              'gbar_Kca1_1_dend_all' : conductvalues[i,10], 
              'gkbar_Kca2_2_dend_trunk' : conductvalues[i,11],
              'gkbar_Kca3_1_dend_trunk' : conductvalues[i,12],
              'gbar_HCN1_dend_all' : conductvalues[i,13],
              'gbar_Nav1_6_soma' : conductvalues[i,14],
              'gbar_Kv1_1_soma' : conductvalues[i,15],
              'gkbar_Kv3_4_soma': conductvalues[i,16],
              'gkbar_Kir2_3_soma': conductvalues[i,17],
              'pcabar_Cav2_1_soma' : conductvalues[i,18],
              'pcabar_Cav3_1_soma' : conductvalues[i,19],
              'gcabar_Cav3_2_soma' : conductvalues[i,20],
              'pcabar_Cav3_3_soma' : conductvalues[i,21],
              'gbar_Kca1_1_soma' : conductvalues[i,22], 
              'gkbar_Kca2_2_soma' : conductvalues[i,23],
              'gkbar_Kca3_1_soma' : conductvalues[i,24],
              'gbar_HCN1_soma' : conductvalues[i,25],
              'gbar_Nav1_6_axon0':0,
              'gkbar_Kv3_4_axon0':conductvalues[i,27],
              'pcabar_Cav2_1_axon0': conductvalues[i,28],
              'pcabar_Cav3_1_axon0' : conductvalues[i,29], 
              'gbar_Kv1_1_axon1':conductvalues[i,30],
              'gbar_Nav1_6_axon':conductvalues[i,31],
              'gkbar_Kv3_4_axon':conductvalues[i,32],
              'pcabar_Cav2_1_axon':conductvalues[i,33],
              'pcabar_Cav3_1_axon' : conductvalues[i,34]} 


    responses = threestep_protocol.run(cell_model=PC2018, param_values=default_params, sim=nrn)
    step1time = responses['step1.soma.v']['time']
    step1vm = responses['step1.soma.v']['voltage']
    sommastep1 = list(zip(step1time,step1vm))
    saveval = str(i)


    np.savetxt(str(saveval) + '_sim_step_noais' + '.txt', sommastep1, delimiter = ' ')

The next cell plots the validated results based on the frequency absence of firing without sodium chanells in the Axon Initial Segment AIS.

In [ ]:
#plot single result.

for i in good_results: #Number of the current step used in the optimization. 1, first step, 2 second step and so on.
    filetxt = str(i) + '_sim_step_noais.txt'

    trace = np.genfromtxt(filetxt)

    time, voltage = zip(*trace)
        
    fig, ax = plt.subplots()
    ax.plot(time, voltage, 'b', label='spikes indiv. %d' % (i))

    legend = ax.legend(loc='upper right', shadow=True)

    frame = legend.get_frame()
    frame.set_facecolor('0.90')

    plt.xlabel("Time (ms)")
    plt.ylabel("Membrane voltage (mv) ")

In [ ]:
#plot single result.
import os

# Location of the recording. For the Purkinje cell there is only one.
location = '' 


for z in good_results:
    for current_inj_step in range (1,4): 
        filetxt = valid_dir + str(z) + '_sim_step' + str(current_inj_step) + location +'.txt'
        if os.path.isfile(filetxt) == False:
            print( 'file not found')
            pass
    
        elif os.path.isfile(filetxt):
            trace = np.genfromtxt(filetxt)

            time, voltage = zip(*trace)
        
            fig, ax = plt.subplots()
            ax.plot(time, voltage, 'b', label='spikes indiv. %d, %s' % (z, steps[current_inj_step]))

            legend = ax.legend(loc='upper right', shadow=True)

            frame = legend.get_frame()
            frame.set_facecolor('0.90')

            plt.xlabel("Time (ms)")
            plt.ylabel("Membrane voltage (mv) ")